In [1]:
import numpy as np
import torch

from torch.nn import functional as F
from torch.utils.data import DataLoader
import util
import geometry
import cv2
import os 
import pdb
import matplotlib.pyplot as plt
from pyntcloud import PyntCloud
from pyntcloud.plot import common as pyntcloud_common
import pandas as pd
import models
import hdf5_dataio
import glob
import data_util
from skimage.metrics import structural_similarity
import open3d as o3d
import matplotlib.colors as pltcolors


import extracting_geometry
import wandb
%load_ext autoreload
%autoreload 2

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


C:\Progra~1\Anaconda\lib\site-packages\requests\__init__.py:89: RequestsDependencyWarning: urllib3 (1.26.3) or chardet (3.0.4) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({}) doesn't match a supported "


In [2]:
checkpoint_path = 'YOUR MODEL CHECKPOINT PATH'
network = 'relu' ##model parameters
conditioning = 'hyper' ## model parameter
data_root = 'LOCATION THE DATASET in hdf5 format'
data_root_images = 'LOCATION OF THE DATASET (IMAGES)'
img_sidelength = 128 ## model parameter
max_num_instances = None ## model parameter
specific_observation_idcs = [44] ## model parameter

In [ ]:
state_dict = torch.load(checkpoint_path)
num_instances = state_dict['latent_codes.weight'].shape[0]

In [ ]:
model = models.LFAutoDecoder(num_instances=num_instances, latent_dim=256, parameterization='plucker', network=network,
                             conditioning=conditioning).cuda()

In [ ]:
model.load_state_dict(state_dict)
model.eval()


In [3]:
"""
    specific_instances is a list of instances, e.g. ['instance_1', 'instance_2',...,'instance_n']
    specific_poses is list of lists, where each list contains the poses you want to use for the point cloud.
    A point cloud is constructed by creating a depth map, converting the 2D coords to 3D coords for each depth map
    stacking those 3D coords such that we have more 3D points and finally construct the point clouds based on those points.
    example of specific_poses = [[1,2,3], [1,2,3],...,[1,2,3]]
    
"""


specific_instances = 'SPECIFY THE INSTANCES YOU WANT TO CREATE DEPTH MAPS AND POINT CLOUDS FOR'
specific_poses = 'SPECIFY THE POSES YOU WANT TO USE FOR THE DEPTH MAPS AND POINT CLOUDS'

assert len(specific_instances) == len(specific_poses) ## for each instance you need a list of poses...

In [ ]:
for instance,poses in zip(specific_instances, specific_poses):
    dataset = hdf5_dataio.SceneClassDataset(num_context=0,
                                                data_root='LOCATION TO YOUR DATA DIRECTORY', 
                                                img_sidelength=img_sidelength,
                                                num_trgt=1,
                                                max_num_instances=1,
                                                specific_observation_idcs=[64],
                                                specific_instances=[instance],
                                                specific_poses=[poses])
    
    dataloader = DataLoader(dataset, batch_size=1, shuffle=False, drop_last=True, num_workers=0)
    output_dir = 'SPECIFY YOUR OUTPUT DIRECTORY'
    depths,points,colors, pose_ids, rgb_ids,images = extracting_geometry.create_depth(dataloader,model)
    util.plot_images(images, output_dir,'prediction')
    extracting_geometry.construct_point_cloud(points,colors, 15.0,125, output_dir,'cloud')
    extracting_geometry.plot_depth_maps(depths, output_dir,'depth')
    util.write_text(pose_ids, output_dir, 'poses_depthmaps')
    util.write_text(rgb_ids, output_dir, 'rgbs_depthmaps')
    